
## Overview

Este notebook tem a função de transformar o arquivo de servidores públicos, extraída do Portal da Transparência do Governo Federal, em tabelas mais normalizadas, as quais possam ser armazenadas no formato PySpark e utilizadas quando necessário.

In [0]:
# File location and type
file_location = "/FileStore/tables/202401_Cadastro.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ";"
encode = 'latin1'

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .option("encoding", encode) \
  .load(file_location)

# display(df)

In [0]:
# Separar tabelas de dados para evitar duplicações desnecessárias
df_cad_servidores = df[["id_SERVIDOR_PORTAL", "NOME", "CPF"]].drop_duplicates(subset=['id_SERVIDOR_PORTAL'])

Cria tabela permanente de cadastro de servidores

In [0]:
# criar tabela de servidores
permanent_table_name = "cadastro_servidores"
df_cad_servidores.write.mode("overwrite").saveAsTable(permanent_table_name)

# df.write.format("parquet").saveAsTable(nome_da_tabela)

In [0]:
%sql
select * from cadastro_servidores limit 5

id_SERVIDOR_PORTAL,NOME,CPF
-11,Sigiloso,null
1025,ANDRE LOPES FULY,***.843.127-**
1030,ANA RITA DANTAS DA SILVEIRA BARROS,***.611.684-**
1031,FLAVIA BRITO MEDINA,***.838.776-**
1088,VILNEY VIEIRA MARCOS,***.795.806-**


In [0]:
# separar unidades organizacionais
df_uorgs = df[["COD_UORG_LOTACAO", "UORG_LOTACAO"]].drop_duplicates()

In [0]:
# criar tabela de unidades organizacionais
permanent_table_name = "unid_org"
# df_uorgs.write.format("parquet").saveAsTable(permanent_table_name)
df_uorgs.write.mode("overwrite").saveAsTable(permanent_table_name)

In [0]:
%sql
select * from unid_org limit 5

COD_UORG_LOTACAO,UORG_LOTACAO
15000000056762,DIV..PESQ.ATEND.MED.ODONT.NUTRICIONAL
44207000001691,NUCLEO GESTAO INTEGRADA - PORTO VELHO
25000000003657,SUS-SMS/SAO GONCALO
26439000000818,DIRETORIA GERAL/CAMPUS AV TUPA
26240000000235,CCA - DPTO SOLOS E ENGENHARIA RURAL


In [0]:
# retirar descrição da UORG do df original
df = df.drop("UORG_LOTACAO")

In [0]:
# separar organização lotação
df_org_lot = df[["COD_ORG_LOTACAO", "ORG_LOTACAO"]].drop_duplicates()

In [0]:
# criar tabela de unidades organizacionais
permanent_table_name = "org_lotacao"
# df_org_lot.write.format("parquet").saveAsTable(permanent_table_name)
df_org_lot.write.mode("overwrite").saveAsTable(permanent_table_name)

In [0]:
%sql
SELECT * FROM org_lotacao LIMIT 5

COD_ORG_LOTACAO,ORG_LOTACAO
26350,Fundação Universidade Federal da Grande Dourados
24204,Instituto do Patrimônio Histórico e Artístico Nacional
26424,Instituto Federal do Tocantins
25211,Casa da Moeda do Brasil
45206,Instituto de Pesquisa Econômica Aplicada


In [0]:
# retirando a descrição do órgão lotação do dataframe principal
df = df.drop("ORG_LOTACAO")

In [0]:
df_org_sup = df[["COD_ORGSUP_LOTACAO", "ORGSUP_LOTACAO"]].drop_duplicates()

In [0]:
permanent_table_name = "org_lotacao_superior"
# df_org_sup.write.format("parquet").saveAsTable(permanent_table_name)
df_org_sup.write.mode("overwrite").saveAsTable(permanent_table_name)

In [0]:
%sql
select * from org_lotacao_superior LIMIT 5

COD_ORGSUP_LOTACAO,ORGSUP_LOTACAO
23000,Ministério da Previdência Social
15000,Ministério da Educação
49200,MINISTERIO DOS TRANSPORTES
17600,MINISTERIO DA FAZENDA
40200,MINISTERIO DAS CIDADES


In [0]:
df = df.drop("ORGSUP_LOTACAO")

In [0]:
cols_to_drop = ['NOME', 'CPF', 'MATRICULA', 'CLASSE_CARGO', 'REFERENCIA_CARGO', 'PADRAO_CARGO', 
               'NIVEL_CARGO', 'SIGLA_FUNCAO', 'NIVEL_FUNCAO', 'CODIGO_ATIVIDADE', 'OPCAO_PARCIAL', 
               'COD_UORG_EXERCICIO', 'UORG_EXERCICIO', 'COD_ORG_EXERCICIO', 'ORG_EXERCICIO', 'COD_ORGSUP_EXERCICIO', 
               'ORGSUP_EXERCICIO', 'COD_TIPO_VINCULO', 'DATA_INICIO_AFASTAMENTO', 'DATA_TERMINO_AFASTAMENTO',
               'REGIME_JURIDICO', 'JORNADA_DE_TRABALHO', 'DATA_INGRESSO_CARGOFUNCAO', 'DATA_NOMEACAO_CARGOFUNCAO', 
               'DOCUMENTO_INGRESSO_SERVICOPUBLICO', 'DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO', 'DIPLOMA_INGRESSO_CARGOFUNCAO',
               'DIPLOMA_INGRESSO_ORGAO', 'DIPLOMA_INGRESSO_SERVICOPUBLICO', 'UF_EXERCICIO'
               'DIPLOMA_INGRESSO_CARGOFUNCAO', 'UF_EXERCICIO'
               ]

In [0]:

# retirar diversas colunas que não são necessárias para respondera as questões
df = df.drop(*cols_to_drop)

In [0]:
permanent_table_name = "qualificacao_servidores"
# df.write.format("parquet").saveAsTable(permanent_table_name)
df.write.mode("overwrite").saveAsTable(permanent_table_name)

In [0]:
%sql
select * from qualificacao_servidores limit 5

Id_SERVIDOR_PORTAL,DESCRICAO_CARGO,FUNCAO,ATIVIDADE,COD_UORG_LOTACAO,COD_ORG_LOTACAO,COD_ORGSUP_LOTACAO,TIPO_VINCULO,SITUACAO_VINCULO,DATA_INGRESSO_ORGAO
3174964,BIBLIOTECARIO-DOCUMENTALISTA,Sem informação,Sem informaç,26405000000040,26405,15000,Cargo,ATIVO PERMANENTE,2008-12-29
2903139,Sem informaç,FUNCAO GRATIFICADA - IFES,DIRETOR(A),26239000001515,26239,15000,Função,ATIVO PERMANENTE,2006-07-24
2903139,PROFESSOR DO MAGISTERIO SUPERIOR,Sem informação,Sem informaç,26239000001515,26239,15000,Cargo,ATIVO PERMANENTE,2006-07-24
2868174,FISIOTERAPEUTA - 30H,Sem informação,Sem informaç,26443035000000,26443,15000,Cargo,CELETISTA/EMPREGADO,2022-12-07
3137242,PROFESSOR ENS BASICO TECN TECNOLOGICO,Sem informação,Sem informaç,26417000000013,26417,15000,Cargo,ATIVO PERMANENTE,2008-12-29


In [0]:
%sql
SELECT a.NOME, b.FUNCAO, c.ORG_LOTACAO  
FROM cadastro_servidores a 
JOIN qualificacao_servidores b ON a.Id_SERVIDOR_PORTAL = b.Id_SERVIDOR_PORTAL 
JOIN org_lotacao c ON b.COD_ORG_LOTACAO = c.COD_ORG_LOTACAO
WHERE b.DESCRICAO_CARGO = 'TECNICO DO SEGURO SOCIAL' AND c.ORG_LOTACAO <> 'Instituto Nacional do Seguro Social' LIMIT 10

NOME,FUNCAO,ORG_LOTACAO
WILMA BOECHAT DE SOUZA SOARES,Sem informação,MINISTERIO DA FAZENDA
MARGARETH ZIMMERMANN,Sem informação,MINISTERIO DA FAZENDA
SOLANGE APARECIDA DO NASCIMENTO SOARES,Sem informação,MINISTERIO DA FAZENDA
CAROLINA BEATRIZ IMTHURN,Sem informação,MINISTERIO DA FAZENDA
PAULO ROBERTO BOTIN,Sem informação,MINISTERIO DA FAZENDA
BRASILIANO DIAS DOS SANTOS,Sem informação,MINISTERIO DA FAZENDA
MARIA LUCIANA PEREIRA DYSMAN SEIXAS,Sem informação,MINISTERIO DA FAZENDA
ROBERTO CARLOS DE ARAUJO,Sem informação,MINISTERIO DA FAZENDA
RICARDO VIANNA JORGE,Sem informação,MINISTERIO DA FAZENDA
VALERIA SILVA FERREIRA DA COSTA,Sem informação,MINISTERIO DA FAZENDA
